## QUEST
- RecurrenceOfSurgery dataset 사용
- 목표변수(재발여부), 설명변수(최소 7개)
- 최고 모델 찾기

In [1]:
import pandas as pd

In [2]:
df_ROS = pd.read_csv('../../../datasets/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


### Pre-Processing

In [4]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

In [3]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [12]:
df_ROS_extract = df_ROS[['재발여부','입원일자','스테로이드치료','연령']]
df_ROS_extract.isnull().sum()

재발여부       0
입원일자       0
스테로이드치료    0
연령         0
dtype: int64

### Scaling & Encoding 
- Encoding with One Hot Encoding : 범주형 데이터를 연속형 숫자 데이터로 변환

In [13]:
df_ROS_extract['스테로이드치료'].value_counts()

1    1134
0     760
Name: 스테로이드치료, dtype: int64

In [16]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder() #인스턴스화 
oneHotEncoder.fit(df_ROS_extract[['스테로이드치료']]) # 학습

OneHotEncoder()

In [17]:
oneHotEncoder.categories_

[array([0, 1], dtype=int64)]

In [21]:
encoded_data = oneHotEncoder.transform(df_ROS_extract[['스테로이드치료']]).toarray()
encoded_data

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [24]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['스테로이드치료']))
df_encoded_data

,스테로이드치료_0,스테로이드치료_1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
1889,0.0,1.0
1890,0.0,1.0
1891,0.0,1.0
1892,0.0,1.0


In [25]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=1894, step=1), (1894, 2))

In [27]:
df_ROS_extract.index, df_ROS_extract.shape # 인덱스 갯수 맞음

(RangeIndex(start=0, stop=1894, step=1), (1894, 4))

### concat(병합)

In [30]:
df_ROS_extract['스테로이드치료'].shape

(1894,)

In [31]:
df_ROS_extract = pd.concat([df_ROS_extract.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_extract

,재발여부,입원일자,스테로이드치료,연령,스테로이드치료_0,스테로이드치료_1
0,0,20190713,1,66,0.0,1.0
1,0,20190715,1,47,0.0,1.0
2,0,20190729,1,39,0.0,1.0
3,0,20190731,0,40,1.0,0.0
4,0,20190903,1,42,0.0,1.0
...,...,...,...,...,...,...
1889,0,20170407,1,59,0.0,1.0
1890,0,20170426,1,42,0.0,1.0
1891,0,20170410,1,61,0.0,1.0
1892,0,20170408,1,29,0.0,1.0


### Scaling - MinMaxScaler

In [32]:
target = df_ROS_extract['재발여부']
features = df_ROS_extract[['입원일자', '연령','스테로이드치료_0','스테로이드치료_1' ]]

In [33]:
features.columns

Index(['입원일자', '연령', '스테로이드치료_0', '스테로이드치료_1'], dtype='object')

In [36]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler() #인스턴스화
features = minMaxScaler.fit_transform(features)

### 정형화 단계

#### split

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111) #random_state=111 랜덤값으로 설정
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((1420, 4), (1420,), (474, 4), (474,))

#### 모델학습

In [39]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
model = DecisionTreeClassifier()
from sklearn.model_selection import GridSearchCV

In [40]:
hyper_params = {'min_samples_leaf' : [5, 7, 9]
               ,'max_depth' : [9, 11]
               ,'min_samples_split' : [5, 6, 7]}

### 평가 score 분류
- Default 값
    - 분류 : 정확도(Accuracy)
    - 예측 : R squre(R-squre)

In [41]:
grid_search = GridSearchCV(model, param_grid=hyper_params, cv=3, verbose=1) 

In [42]:
grid_search.fit(features_train, target_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [9, 11], 'min_samples_leaf': [5, 7, 9],
                         'min_samples_split': [5, 6, 7]},
             verbose=1)

In [43]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=9, min_samples_leaf=9, min_samples_split=5)

In [49]:
grid_search.best_score_, grid_search.best_params_

## best_score : 0.868 --> 데이터 정확도가 비교적 높음을 확인

(0.8683032860248051,
 {'max_depth': 9, 'min_samples_leaf': 9, 'min_samples_split': 5})

In [45]:
best_model = grid_search.best_estimator_ # 하나의 모델 --> 그 중에서 최고의 모델
best_model  

DecisionTreeClassifier(max_depth=9, min_samples_leaf=9, min_samples_split=5)

In [46]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 평가

In [47]:
from sklearn.metrics import classification_report

In [48]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93       416
           1       0.00      0.00      0.00        58

    accuracy                           0.87       474
   macro avg       0.44      0.50      0.47       474
weighted avg       0.77      0.87      0.82       474

